In [94]:
import pandas as pd
df = pd.read_csv('portales_empleo.csv')

In [95]:

df.columns

Index(['Unnamed: 0', 'Título', 'Ubicación', 'Descripción', 'Servicios',
       'Sueldo', 'Timestamp', 'Url'],
      dtype='object')

In [75]:
print(df.columns.tolist())


['Unnamed: 0', 'Título', 'Ubicación', 'Descripción', 'Servicios', 'Timestamp', 'Url', 'Sueldo medio']


In [84]:
# Función para limpiar el salario:
def limpieza_salario(rango_salarios):
    rango_salarios = rango_salarios.replace('EUR','').replace(',','')
    rango = rango_salarios.split(' - ')

    salario_min = int(rango[0])
    salario_max = int(rango[1])

    #Devolvemos la media
    return (salario_max+salario_min)//2

df['Sueldo medio'] = df['Sueldo'].apply(limpieza_salario)
df = df.drop(columns='Sueldo')

In [90]:

df['Sueldo medio']

KeyError: 'Sueldo medio'

Entramos al apartado de limpieza y preparación de SQL

In [30]:
!pip install scikit-learn

In [31]:
!pip install plotly

In [33]:
import sys
print(sys.executable)

/usr/local/bin/python3


In [34]:
pip uninstall scikit-learn

Note: you may need to restart the kernel to use updated packages.


In [35]:
pip install scikit-learn

  Using cached scikit_learn-1.5.2-cp312-cp312-macosx_12_0_arm64.whl.metadata (13 kB)
  Using cached joblib-1.4.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached threadpoolctl-3.5.0-py3-none-any.whl.metadata (13 kB)
Using cached scikit_learn-1.5.2-cp312-cp312-macosx_12_0_arm64.whl (11.0 MB)
Using cached joblib-1.4.2-py3-none-any.whl (301 kB)
Using cached threadpoolctl-3.5.0-py3-none-any.whl (18 kB)

[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [37]:
pip uninstall plotly

Note: you may need to restart the kernel to use updated packages.


In [38]:
pip install plotly

  Using cached plotly-5.24.1-py3-none-any.whl.metadata (7.3 kB)
Using cached plotly-5.24.1-py3-none-any.whl (19.1 MB)

[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [40]:
pip install nbformat

  Using cached nbformat-5.10.4-py3-none-any.whl.metadata (3.6 kB)
  Using cached fastjsonschema-2.20.0-py3-none-any.whl.metadata (2.1 kB)
  Using cached jsonschema-4.23.0-py3-none-any.whl.metadata (7.9 kB)
  Using cached referencing-0.35.1-py3-none-any.whl.metadata (2.8 kB)
  Using cached rpds_py-0.20.0-cp312-cp312-macosx_11_0_arm64.whl.metadata (4.2 kB)
Using cached nbformat-5.10.4-py3-none-any.whl (78 kB)
Using cached fastjsonschema-2.20.0-py3-none-any.whl (23 kB)
Using cached jsonschema-4.23.0-py3-none-any.whl (88 kB)
Using cached referencing-0.35.1-py3-none-any.whl (26 kB)
Using cached rpds_py-0.20.0-cp312-cp312-macosx_11_0_arm64.whl (313 kB)

[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, StaleElementReferenceException
from bs4 import BeautifulSoup
import pandas as pd
from time import sleep
import requests
import sqlite3
import re
from sklearn.cluster import KMeans
import plotly.io as pio
import plotly.express as px
import plotly.graph_objects as go
import nbformat
# Lee el archivo CSV
df = pd.read_csv('portales_empleo.csv')

# Muestra las primeras filas del DataFrame
print(df.head())

df = df.drop(columns=['Unnamed: 0'])
# Eliminar filas duplicadas en su totalidad
df = df.drop_duplicates()
df.info()

df.head(5)

# Función para limpiar el salario:
def limpieza_salario(rango_salarios):
    rango_salarios = rango_salarios.replace('EUR','').replace(',','')
    rango = rango_salarios.split(' - ')

    salario_min = int(rango[0])
    salario_max = int(rango[1])

    #Devolvemos la media
    return (salario_max+salario_min)//2

df['Sueldo medio'] = df['Sueldo'].apply(limpieza_salario)
df = df.drop(columns='Sueldo')


# Elimina el texto entre paréntesis en la columna 'título'
df['Título'] = df['Título'].str.replace(r'\(.*?\)', '', regex=True)

# Elimina los signos '|' y '/'
df['Título'] = df['Título'].str.replace(r'[|/]', '', regex=True)

# Elimina espacios en blanco adicionales
df['Título'] = df['Título'].str.strip()

# Elimina el texto entre corchetes en la columna 'título'
df['Título'] = df['Título'].str.replace(r'\[.*?\]', '', regex=True)

# Elimina los signos '|', '/' y '-'
df['Título'] = df['Título'].str.replace(r'[|/-]', '', regex=True)

# Elimina espacios en blanco adicionales
df['Título'] = df['Título'].str.strip()

# Muestra las primeras filas del DataFrame
df.head(5)
# Función para actualizar títulos
def actualizar_titulo(titulo):
    # Reemplazar 'jr' por 'Junior' y 'sr' por 'Senior'
    titulo = titulo.replace(' jr ', ' Junior ').replace(' sr ', ' Senior ')
    titulo = titulo.replace('jr ', 'Junior ').replace('sr ', 'Senior ')
    titulo = titulo.replace(' jr', ' Junior').replace(' sr', ' Senior')
    return titulo

# Aplicar la función de actualización a la columna 'Título'
df['Título'] = df['Título'].apply(actualizar_titulo)
def get_expertise(title):
    if 'Senior' in title or 'senior' in title or 'master' in title or 'manager' in title:
        return 'Senior'
    elif 'Junior' in title or 'junior' in title:
        return 'Junior'
    elif 'intern' in title or 'becario' in title:
        return 'Intern'
    elif 'Lead' in title or 'lead' in title:
        return 'Lead' 
    else:
        return 'Regular'

# Crear la nueva columna "Expertise" aplicando la función sobre la columna "Título"
df['Expertise'] = df['Título'].apply(get_expertise)
import re
# Listas de palabras clave (nombres de empresas y sectores)
empresas = ['santander', 'amazon', 'microsoft', 'vie', 'bcg', 'mckinsey']  # Añadir más nombres de empresas
sectores = ['marketing', 'medtech', 'fintech', 'ecommerce', 'finance', 'game','genai','digital', 'tax', 'governance', 'energy' ]  # Añadir más sectores

# Convertir las listas a un conjunto de expresiones regulares para facilitar la búsqueda
regex_empresas = re.compile(r'\b(' + '|'.join(empresas) + r')\b', re.IGNORECASE)
regex_sectores = re.compile(r'\b(' + '|'.join(sectores) + r')\b', re.IGNORECASE)

# Función para extraer empresa
def extraer_empresa(titulo):
    match = regex_empresas.search(titulo)
    return match.group(0) if match else None

# Función para extraer sector
def extraer_sector(titulo):
    match = regex_sectores.search(titulo)
    return match.group(0) if match else None

# Función para limpiar el título
def limpiar_titulo(titulo):
    # Eliminar nombres de empresas y sectores del título
    titulo = regex_empresas.sub('', titulo)
    titulo = regex_sectores.sub('', titulo)
    # Eliminar espacios extra
    return ' '.join(titulo.split())

# Aplicar las funciones a las columnas del DataFrame
df['Empresa'] = df['Título'].apply(extraer_empresa)
df['Sector'] = df['Título'].apply(extraer_sector)
df['Título'] = df['Título'].apply(limpiar_titulo)
df.head(12)
# Detecta la palabra clave "Inglés" y asigna a la nueva columna 'Otro Idioma'
df['Otro Idioma'] = df['Título'].apply(lambda x: 'inglés' if 'inglés' in x else None)

# Detecta la palabra clave "Inglés" y asigna a la nueva columna 'Otro Idioma'
df['Otro Idioma'] = df['Título'].apply(lambda x: 'English' if 'English' in x else None)

# Detecta las palabras clave "Híbrido", "Remoto" o "Presencial" y asigna a la nueva columna 'Modalidad'
df['Modalidad'] = df['Título'].apply(lambda x: 'Híbrido' if 'Híbrido' in x else ('Remoto' if 'Remoto' in x else ('Presencial' if 'Presencial' in x else None)))

df['Otro Idioma'] = df['Título'].apply(lambda x: 'english' if 'english' in x else None)
df['Otro Idioma'] = df['Título'].apply(lambda x: 'french' if 'french' in x else None)
df['Otro Idioma'] = df['Título'].apply(lambda x: 'francés' if 'francés' in x else None)
# Función para detectar los idiomas
def detectar_idiomas(titulo):
    if 'inglés' in titulo.lower() or 'english' in titulo.lower():
        return 'Inglés'
    elif 'francés' in titulo.lower() or 'french' in titulo.lower():
        return 'Francés'
    # Puedes añadir más idiomas aquí
    return None

# Aplicar la función para detectar idiomas
df['Otro Idioma'] = df['Título'].apply(detectar_idiomas)
titulos_agrupados = df['Otro Idioma'].value_counts()
print(titulos_agrupados)
# Diccionario de ubicaciones y sus modificaciones
ubicaciones_dict = {
    'Boadilla del Monte': 'Madrid',
    'Alcobendas': 'Madrid',
    'Leganés': 'Madrid',
    'Getafe': 'Madrid',
    'Barcelona, Centro': 'Cataluña',
    'Esplugues de Llobregat': 'Cataluña',
    'Barcelona': 'Cataluña',
    'Pozuelo de Alarcón': 'Madrid',
    'Tres Cantos': 'Madrid',
    'Villaviciosa de Odón': 'Madrid',
    'Sant Just Desvern': 'Cataluña'
}

# Función para reemplazar la ubicación según el diccionario
df['Ubicación'] = df['Ubicación'].replace(ubicaciones_dict)
titulos_agrupados = df['Ubicación'].value_counts()
print(titulos_agrupados)
titulos_agrupados = df['Empresa'].value_counts()
print(titulos_agrupados)
# Remover caracteres innecesarios y estandarizar
df['Título'] = df['Título'].str.replace(r'[^\w\s]', '', regex=True).str.strip().str.lower()
# Crear columna de 'Puesto' principal basado en palabras clave
def clasificar_rol(titulo):
    if 'data scientist' in titulo:
        return 'Data Scientist'
    elif 'data applied scientist ii at barcelona spain' in titulo:
        return 'Data Scientist'
    elif 'data engineer' in titulo:
        return 'Data Engineer'
    elif 'analyst' in titulo:
        return 'Data Analyst'
    elif 'machine learning' in titulo:
        return 'Machine Learning Engineer'
    elif 'business intelligence' in titulo:
        return 'Business Intelligence'
    elif 'consultant' in titulo:
        return 'Consultant'
    elif 'specialist' in titulo:
        return 'Data Specialist'
    else:
        return 'Other'

df['Puesto'] = df['Título'].apply(clasificar_rol)
# Reordenar columnas en el orden especificado
columnas_ordenadas = [
    'Título', 'Puesto', 'Sueldo medio','Expertise', 'Empresa', 'Modalidad', 'Sector', 
    'Ubicación', 'Descripción', 'Servicios', 'Otro Idioma'
]
df = df[columnas_ordenadas]
# Función para clasificar el puesto basado en palabras clave extendidas
def clasificar_rol(titulo):
    titulo = titulo.lower()
    if 'data scientist' in titulo:
        return 'Data Scientist'
    elif 'data applied scientist ii at barcelona spain' in titulo:
        return 'Data Scientist'
    elif 'data engineer' in titulo:
        return 'Data Engineer'
    elif 'data manager' in titulo or 'manager data' in titulo:
        return 'Data Manager'
    elif 'head of data' in titulo:
        return 'Data Manager'
    elif 'data expert' in titulo:
        return 'Data Expert'
    elif 'big data architect' in titulo:
        return 'Big Data Architect'
    elif 'manager data cloud' in titulo:
        return 'Data Manager Cloud'
    elif 'data analytics' in titulo:
        return 'Data Analytics'
    elif 'ai innovation architect' in titulo:
        return 'AI Innovation Architect'
    elif 'data management engineer' in titulo:
        return 'Data Manager'
    elif 'business intelligence' in titulo:
        return 'Business Intelligence'
    elif 'analyst' in titulo:
        return 'Data Analyst'
    elif 'consultant' in titulo:
        return 'Consultant'
    elif 'machine learning' in titulo:
        return 'Machine Learning Engineer'
    elif 'specialist' in titulo:
        return 'Data Specialist'
    elif 'data developer' in titulo:
        return 'Data Developer'
    elif 'project manager' in titulo:
        return 'Project Manager'
    else:
        return 'Other'

# Aplicar la función a la columna "Título" para crear o actualizar la columna "Puesto"
df['Puesto'] = df['Título'].apply(clasificar_rol)

# Función para limpiar nombres de ciudades y códigos de puesto directamente en "Título"
def limpiar_titulo(titulo):
    # Eliminación de nombres de ciudades comunes en títulos
    ciudades = ["barcelone", "alicante", "catalonia", "madrid", "bilbao"]  # Añadir otras ciudades según sea necesario
    for ciudad in ciudades:
        titulo = re.sub(rf"\b{ciudad}\b", "", titulo, flags=re.IGNORECASE)
    
    # Eliminación de códigos de puesto que suelen ser letras y números juntos
    titulo = re.sub(r"\b[a-zA-Z]{2,3}\d{3}\b", "", titulo)
    
    # Eliminación de espacios extras generados por las sustituciones
    titulo = re.sub(r"\s+", " ", titulo).strip()
    return titulo

# Aplicar la función de limpieza a la columna "Título" para modificarla directamente
df['Título'] = df['Título'].apply(limpiar_titulo)

# Función para extraer herramientas tecnológicas
def extraer_herramientas(titulo):
    herramientas = ["Salesforce", "CRM", "Power BI", "Qlik", "Python", "Tableau", "SAS", "SAP", "BigQuery", "Datastage", "Analytics", "Pentaho", "Spark"]  # Añadir otras herramientas según sea necesario
    herramientas_encontradas = [herramienta for herramienta in herramientas if herramienta.lower() in titulo.lower()]
    return ", ".join(herramientas_encontradas) if herramientas_encontradas else None

# Crear la columna "Herramientas" con las herramientas tecnológicas encontradas en el título
df['EntornoTEC'] = df['Título'].apply(extraer_herramientas)
def convertir_abreviaciones(titulo):
    titulo = titulo.lower()  # Convertir a minúsculas para comparación
    if 'sr' in titulo:
        titulo = titulo.replace('sr', 'senior')
    if 'jr' in titulo:
        titulo = titulo.replace('jr', 'junior')
    return titulo

# Aplicar la función para convertir los títulos
df['Título'] = df['Título'].apply(convertir_abreviaciones)
def get_expertise(title):
    if 'Senior' in title or 'senior' in title or 'master' in title or 'manager' in title:
        return 'Senior'
    elif 'Junior' in title or 'junior' in title or 'associate' in title:
        return 'Junior'
    elif 'intern' in title or 'becario' in title or 'becarioa' in title:
        return 'Intern'
    elif 'Lead' in title or 'lead' in title or 'head' in title:
        return 'Lead' 
    else:
        return 'Regular'

# Crear la nueva columna "Expertise" aplicando la función sobre la columna "Título"
df['Expertise'] = df['Título'].apply(get_expertise)
titulos_agrupados = df['Puesto'].value_counts()
# Filtrar filas que contengan el texto deseado en la columna "Título"
filtro = df[df['Puesto'].str.contains("Other", case=False, na=False)]

# Función para agregar espacios entre palabras correctamente
def add_spaces(text):
    # Añade un espacio entre palabras que estén pegadas (ej: "SnacksWeekly" -> "Snacks Weekly")
    corrected_text = re.sub(r'(?<=[a-zA-Z])(?=[A-Z])', ' ', text)
    return corrected_text

# Aplicar la función a la columna "Servicios"
df['Servicios'] = df['Servicios'].apply(add_spaces)
# Lista de palabras comunes a eliminar
common_words = r'\b(de|a|in|or|and|the|of|en|con|para|por|y|del|la|los|las|un|una)\b'

# Definición de las palabras clave para cada categoría
benefits_keywords = r'\b(Tarjeta de restaurante|Teletrabajo opcional|Flexible hours|Modern office environment|Continuous learning programs|Mentoring programs|Collaborative culture|Oportunidades de crecimiento profesional|Paquete competitivo de compensación y beneficios|Retirement program|Well-being programs|Diversity initiatives|Work-life balance|Remote work)\b'
skills_keywords = r'\b(Neural networks|Bayesian modeling|Data storytelling|Análisis de datos|Métodos estadísticos|Business Intelligence|Data Science|Machine Learning|Deep Learning|Data Visualization|Problem Solving|Data Analysis|Reporting|Data governance|Statistical Analysis|Predictive Modeling)\b'
tools_keywords = r'\b(Python|R|Pyspark|Power BI|Tableau|Databricks|GCP|SAS|SQL|Excel|VBA|Google Analytics|Adobe Analytics|Domo|Snowflake|TensorFlow|Keras|Scikit-learn|Power Platforms|Jupyter Notebooks|Azure|AWS|Docker|GitHub|Streamlit|Pytorch|BigQuery|SQL)\b'
education_keywords = r'\b(Degree in Computing or Statistics|Bachelor\'s Degree|Master\'s Degree|PhD|Licenciatura|Grado|Engineering Degree|Data Science Degree|Statistics Degree|Computer Science Degree|Mathematics Degree)\b'

# Función para limpiar texto de palabras comunes
def remove_common_words(text):
    return re.sub(common_words, '', text, flags=re.IGNORECASE)

# Función para clasificar las palabras en las categorías
def classify_services(services):
    # Limpiar palabras comunes
    services_cleaned = remove_common_words(services)
    
    # Clasificar términos en categorías
    benefits = re.findall(benefits_keywords, services_cleaned, flags=re.IGNORECASE)
    skills = re.findall(skills_keywords, services_cleaned, flags=re.IGNORECASE)
    tools = re.findall(tools_keywords, services_cleaned, flags=re.IGNORECASE)
    education = re.findall(education_keywords, services_cleaned, flags=re.IGNORECASE)
    
    # Devolver resultados en columnas separadas
    return pd.Series({
        'Beneficios': ', '.join(benefits) if benefits else None,
        'Habilidades': ', '.join(skills) if skills else None,
        'Herramientas': ', '.join(tools) if tools else None,
        'Educación': ', '.join(education) if education else None,
    })

# Aplicar la función a cada fila
df_new = df['Servicios'].apply(classify_services)






   Unnamed: 0                                       Título Ubicación  \
0           0                                 Data Analyst    Madrid   
1           1  Business Intelligence Analyst (all genders)    Madrid   
2           2   Research Analyst (6 month Maternity Cover)    Madrid   
3           3    Senior Program Manager, Analyst Relations    Madrid   
4           4                       [RJ072] | Data Analyst    Madrid   

                                         Descripción  \
0  Una empresa internacional y dinámica busca un ...   
1  An innovative edtech company is seeking a Busi...   
2  An established industry player is seeking a Re...   
3  Join a forward-thinking company at the forefro...   
4  Join a forward-thinking company as a Data Anal...   

                                           Servicios               Sueldo  \
0  ['Tarjeta de restauranteSeguro de saludSeguro ...  EUR 50,000 - 70,000   
1  ['Flexible hybrid work environmentComprehensiv...  EUR 50,000 - 70,000   

In [2]:
df.columns

Index(['Título', 'Puesto', 'Sueldo medio', 'Expertise', 'Empresa', 'Modalidad',
       'Sector', 'Ubicación', 'Descripción', 'Servicios', 'Otro Idioma',
       'EntornoTEC'],
      dtype='object')

In [8]:
df_new

,Beneficios,Habilidades,Herramientas,Educación
0,None,None,"Excel, R, Python","Licenciatura, Licenciatura, Licenciatura, Lice..."
1,None,"Data Governance, Data Science",Tableau,Bachelor's Degree
2,None,"Data Analysis, Data Visualization","Excel, Tableau, Excel, Tableau",None
3,None,None,None,Bachelor's Degree
4,None,"Data Analysis, Data Visualization","Excel, R, Python","Bachelor's degree, Bachelor's degree, Bachelor..."
...,...,...,...,...
1038,None,"Machine learning, Data analysis",None,None
1039,None,"Machine Learning, Data Science, Data visualiza...","Python, Azure",Master's Degree
1040,None,None,None,None
1041,None,None,None,None


In [40]:
df_final = pd.concat([df,df_new],axis=0,ignore_index=True)

df_final = df_final.dropna(subset=['Título'])

In [9]:
df_final = df_final.drop_duplicates()

In [27]:
df_final.columns

Index(['Título', 'Puesto', 'Sueldo medio', 'Expertise', 'Empresa', 'Modalidad',
       'Sector', 'Ubicación', 'Descripción', 'Servicios', 'Otro_Idioma',
       'EntornoTEC', 'Beneficios', 'Habilidades', 'Herramientas', 'Educación',
       'Sueldo_medio'],
      dtype='object')

In [ ]:
df_final['Sueldo_medio'] = df_final['Sueldo medio']
df_final.drop(columns='Sueldo medio')

In [28]:
df_final.drop(columns='Sueldo medio')

,Título,Puesto,Expertise,Empresa,Modalidad,Sector,Ubicación,Descripción,Servicios,Otro_Idioma,EntornoTEC,Beneficios,Habilidades,Herramientas,Educación,Sueldo_medio
0,data analyst,Data Analyst,Regular,None,None,None,Madrid,Una empresa internacional y dinámica busca un ...,['Tarjeta de restaurante Seguro de salud Segur...,None,None,None,None,"Excel, R, Python","Licenciatura, Licenciatura, Licenciatura, Lice...",60000
1,business intelligence analyst,Business Intelligence,Regular,None,None,None,Madrid,An innovative edtech company is seeking a Busi...,['Flexible hybrid work environment Comprehensi...,None,None,None,"Data Governance, Data Science",Tableau,Bachelor's Degree,60000
2,research analyst,Data Analyst,Regular,None,None,None,Madrid,An established industry player is seeking a Re...,['Competitive benefits Career development oppo...,None,None,None,"Data Analysis, Data Visualization","Excel, Tableau, Excel, Tableau",None,40000
3,senior program manager analyst relations,Data Analyst,Senior,None,None,None,Madrid,Join a forward-thinking company at the forefro...,['Equity (Restricted Stock Units)Life Insuranc...,None,None,None,None,None,Bachelor's Degree,60000
4,data analyst,Data Analyst,Regular,None,None,None,Madrid,Join a forward-thinking company as a Data Anal...,['Flexible working hours Health insurance Life...,None,None,None,"Data Analysis, Data Visualization","Excel, R, Python","Bachelor's degree, Bachelor's degree, Bachelor...",60000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1038,senior data analyst,Data Analyst,Senior,None,None,None,Cataluña,An innovative digital media company is seeking...,['Competitive salary Travel opportunities Full...,None,None,None,"Machine learning, Data analysis",None,None,60000
1039,data scientist barcelona sep 27 2024,Data Scientist,Regular,None,None,None,Cataluña,Únete a una aerolínea innovadora que conecta c...,['Membresía de gimnasio con descuento Fruta fr...,None,None,None,"Machine Learning, Data Science, Data visualiza...","Python, Azure",Master's Degree,60000
1040,data analyst,Data Analyst,Regular,None,None,None,Cataluña,Sin descripción,[],None,None,None,None,None,None,60000
1041,junior data analyst based in barcelona,Data Analyst,Junior,None,None,Finance,Cataluña,Sin descripción,[],None,None,None,None,None,None,40000


In [29]:
# Combinar los resultados en un nuevo DataFrame
df_final = pd.concat([df, df_new], axis=1)

df_final.to_csv("Datos_definitivos.csv", index = False)
# Cargar los datos desde un archivo CSV
data = pd.read_csv('Datos_definitivos.csv')

# Verificar las primeras filas de los datos
print(data.head())

# Preparar los datos para el clustering (selecciona las características relevantes)
X = data[['Puesto', 'Expertise', 'Sector']]

# Convertir variables categóricas a numéricas
X['Puesto'] = X['Puesto'].astype('category').cat.codes
X['Expertise'] = X['Expertise'].astype('category').cat.codes
X['Sector'] = X['Sector'].astype('category').cat.codes

# Aplicar K-means para segmentar los datos
kmeans = KMeans(n_clusters=3, random_state=0)
data['Segmento'] = kmeans.fit_predict(X)

# Visualizar la cantidad de ofertas por segmento
segment_counts = data['Segmento'].value_counts().reset_index()
segment_counts.columns = ['Segmento', 'Cantidad']

# Gráfico de barras mostrando la cantidad de ofertas por segmento
fig_barras = go.Figure(data=[
    go.Bar(x=segment_counts['Segmento'], y=segment_counts['Cantidad'], 
           marker=dict(color='royalblue'))
])

# Configurar el layout del gráfico de barras
fig_barras.update_layout(
    title='Cantidad de Ofertas por Segmento',
    xaxis_title='Segmento',
    yaxis_title='Cantidad de Ofertas'
)

# Guardar el gráfico como archivo HTML y abrirlo en el navegador
pio.write_html(fig_barras, 'grafico_barras.html')
import webbrowser
webbrowser.open('grafico_barras.html')

# Gráfico de dispersión mostrando los diferentes segmentos
fig_dispersion = go.Figure()

# Agregar trazas para cada segmento
for segmento in data['Segmento'].unique():
    seg_data = data[data['Segmento'] == segmento]
    fig_dispersion.add_trace(go.Scatter(
        x=seg_data['Puesto'], 
        y=seg_data['Expertise'], 
        mode='markers',
        name=f'Segmento {segmento}',
        text=seg_data['Descripción'],  # Mostrar descripción al pasar el ratón
        marker=dict(size=10)
    ))

# Configurar el layout del gráfico de dispersión
fig_dispersion.update_layout(
    title='Resultados del Análisis de Segmentación',
    xaxis_title='Puesto',
    yaxis_title='Expertise',
    legend_title='Segmentos',
    hovermode='closest'
)

# Guardar el gráfico de dispersión como archivo HTML y abrirlo en el navegador
pio.write_html(fig_dispersion, 'grafico_dispersion.html')
webbrowser.open('grafico_dispersion.html')
# Asegurarte de que el DataFrame tiene los nombres de columnas correctos
df_final.columns = [
    "Título", "Puesto", 'Sueldo_medio',"Expertise", "Empresa", "Modalidad", "Sector", 
    "Ubicación", "Descripción", "Servicios", "Otro_Idioma", "EntornoTEC", 
    "Beneficios", "Habilidades", "Herramientas", "Educación"
]


                                     Título                 Puesto  \
0                              data analyst           Data Analyst   
1             business intelligence analyst  Business Intelligence   
2                          research analyst           Data Analyst   
3  senior program manager analyst relations           Data Analyst   
4                              data analyst           Data Analyst   

   Sueldo medio Expertise Empresa Modalidad Sector Ubicación  \
0         60000   Regular     NaN       NaN    NaN    Madrid   
1         60000   Regular     NaN       NaN    NaN    Madrid   
2         40000   Regular     NaN       NaN    NaN    Madrid   
3         60000    Senior     NaN       NaN    NaN    Madrid   
4         60000   Regular     NaN       NaN    NaN    Madrid   

                                         Descripción  \
0  Una empresa internacional y dinámica busca un ...   
1  An innovative edtech company is seeking a Busi...   
2  An established industry

/var/folders/5s/6q7y_kyj6zl53g0f3jycmzsr0000gn/T/ipykernel_71898/37786944.py:15: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/5s/6q7y_kyj6zl53g0f3jycmzsr0000gn/T/ipykernel_71898/37786944.py:16: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/5s/6q7y_kyj6zl53g0f3jycmzsr0000gn/T/ipykernel_71898/37786944.py:17: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the docu

In [39]:
df_final.columns

Index(['Título', 'Puesto', 'Sueldo_medio', 'Expertise', 'Empresa', 'Modalidad',
       'Sector', 'Ubicación', 'Descripción', 'Servicios', 'Otro_Idioma',
       'EntornoTEC', 'Beneficios', 'Habilidades', 'Herramientas', 'Educación'],
      dtype='object')

In [ ]:

# Conectar a la base de datos SQLite
conn = sqlite3.connect('database.db')
cursor = conn.cursor()
# Eliminamos y creamos la tabla
cursor.execute("DROP TABLE IF EXISTS bbdd_ofertas")
cursor.execute('''
    CREATE TABLE bbdd_ofertas (
        Título TEXT,
        Puesto TEXT,
        Expertise TEXT,
        Empresa TEXT,
        Modalidad TEXT,
        Sueldo_medio TEXT,
        Sector TEXT,
        Ubicación TEXT,
        Descripción TEXT,
        Servicios TEXT,
        Otro_Idioma TEXT,
        EntornoTEC TEXT,
        Beneficios TEXT,
        Habilidades TEXT,
        Herramientas TEXT,
        Educación TEXT
    )
''')


# Asegúrate de que el DataFrame tiene los nombres de columnas correctos
print(df_final.columns)  # Verificar los nombres de las columnas

# Insertar el DataFrame en la tabla de SQLite
df_final.to_sql('bbdd_ofertas', conn, if_exists='append', index=False)

# Verificar que los datos se hayan guardado correctamente
df_check = pd.read_sql("SELECT * FROM bbdd_ofertas LIMIT 5", conn)
print(df_check)

# Cerrar la conexión
conn.close()


# Conectar a la base de datos SQLite
conn = sqlite3.connect('database.db')

# Leer los datos de la tabla 'job_data' en un DataFrame de pandas
query = "SELECT * FROM bbdd_ofertas LIMIT 5"
df_check = pd.read_sql(query, conn)

# Mostrar el DataFrame para verificar que los datos se hayan guardado
print(df_check)

# Cerrar la conexión
conn.close()
 # Conectar a la base de datos SQLite
conn = sqlite3.connect('database.db')

# Consultar los datos donde la ubicación es "Madrid"
query = "SELECT * FROM bbdd_ofertas WHERE Ubicación = 'Madrid'"
df_madrid = pd.read_sql(query, conn)

# Mostrar el DataFrame con los registros de Madrid
print(df_madrid)

# Cerrar la conexión
conn.close()

Index(['Título', 'Puesto', 'Sueldo_medio', 'Expertise', 'Empresa', 'Modalidad',
       'Sector', 'Ubicación', 'Descripción', 'Servicios', 'Otro_Idioma',
       'EntornoTEC', 'Beneficios', 'Habilidades', 'Herramientas', 'Educación'],
      dtype='object')
                                     Título                 Puesto Expertise  \
0                              data analyst           Data Analyst   Regular   
1             business intelligence analyst  Business Intelligence   Regular   
2                          research analyst           Data Analyst   Regular   
3  senior program manager analyst relations           Data Analyst    Senior   
4                              data analyst           Data Analyst   Regular   

  Empresa Modalidad Sueldo_medio Sector Ubicación  \
0    None      None        60000   None    Madrid   
1    None      None        60000   None    Madrid   
2    None      None        40000   None    Madrid   
3    None      None        60000   None    Madrid   

In [33]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1043 entries, 0 to 1042
Data columns (total 16 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Título        1043 non-null   object
 1   Puesto        1043 non-null   object
 2   Sueldo_medio  1043 non-null   int64 
 3   Expertise     1043 non-null   object
 4   Empresa       24 non-null     object
 5   Modalidad     14 non-null     object
 6   Sector        87 non-null     object
 7   Ubicación     1043 non-null   object
 8   Descripción   1043 non-null   object
 9   Servicios     1043 non-null   object
 10  Otro_Idioma   27 non-null     object
 11  EntornoTEC    86 non-null     object
 12  Beneficios    76 non-null     object
 13  Habilidades   379 non-null    object
 14  Herramientas  646 non-null    object
 15  Educación     381 non-null    object
dtypes: int64(1), object(15)
memory usage: 130.5+ KB


In [42]:
df_final['Puesto'].value_counts()

Puesto
Data Analyst                 435
Data Engineer                235
Data Scientist               198
Other                         73
Machine Learning Engineer     32
Data Specialist               28
Business Intelligence         16
Data Analytics                 9
Consultant                     8
Data Developer                 4
Project Manager                2
Data Expert                    2
AI Innovation Architect        1
Name: count, dtype: int64

In [35]:
df_final['Título'].value_counts()

Título
data engineer                                                64
data scientist                                               64
data analyst                                                 51
senior data scientist                                        28
senior data engineer                                         28
                                                             ..
s944 senior business analyst                                  1
security analyst penetration tester barcelona spainremote     1
senior analyst center of financial insight a194               1
prime business analyst                                        1
product operations lead                                       1
Name: count, Length: 498, dtype: int64

In [43]:
df_final['Sueldo_medio'].describe()

KeyError: 'Sueldo_medio'